In [ ]:
!pip install tsplib95
!pip install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.2 which is in

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import zipfile
import tsplib95
from tqdm import tqdm

In [ ]:
file_path = 'tsp.zip'
folder_path = ''

In [ ]:
with zipfile.ZipFile(file_path, 'r') as zip_ref:
  zip_ref.extractall(folder_path)
  instances = zip_ref.namelist()

In [ ]:
k_factor_1 = 1/4
k_factor_2 = 1/2
k_factor_3 = 3/4

In [ ]:
# instances
instances = [
  'tsp/a280.tsp', 'tsp/bayg29.tsp', 'tsp/bays29.tsp', 'tsp/berlin52.tsp', 'tsp/bier127.tsp',
  'tsp/burma14.tsp', 'tsp/ch130.tsp', 'tsp/ch150.tsp', 'tsp/d198.tsp', 'tsp/dantzig42.tsp',
  'tsp/eil101.tsp', 'tsp/eil51.tsp', 'tsp/eil76.tsp', 'tsp/fri26.tsp', 'tsp/gil262.tsp',
  'tsp/gr137.tsp', 'tsp/gr17.tsp', 'tsp/gr21.tsp', 'tsp/gr24.tsp', 'tsp/gr48.tsp',
  'tsp/gr96.tsp', 'tsp/gr202.tsp', 'tsp/gr229.tsp', 'tsp/hk48.tsp', 'tsp/kroA100.tsp',
  'tsp/kroA150.tsp', 'tsp/kroA200.tsp', 'tsp/kroB100.tsp', 'tsp/kroB150.tsp',
  'tsp/kroB200.tsp', 'tsp/kroC100.tsp', 'tsp/kroD100.tsp', 'tsp/kroE100.tsp',
  'tsp/lin105.tsp', 'tsp/lin318.tsp', 'tsp/pr107.tsp', 'tsp/pr124.tsp', 'tsp/pr136.tsp',
  'tsp/pr144.tsp', 'tsp/pr152.tsp', 'tsp/pr76.tsp', 'tsp/pr226.tsp', 'tsp/rat195.tsp',
  'tsp/rat99.tsp', 'tsp/rd100.tsp', 'tsp/st70.tsp', 'tsp/swiss42.tsp', 'tsp/u159.tsp',
  'tsp/ulysses16.tsp', 'tsp/ulysses22.tsp'
]

selected_instances = ['tsp/burma14.tsp', 'tsp/ulysses16.tsp', 'tsp/gr24.tsp', 'tsp/dantzig42.tsp', 'tsp/eil76.tsp', 'tsp/lin318.tsp']

In [ ]:
def get_instance(instance_name):
  problem = tsplib95.load(instance_name)
  problem_name = problem.name
  problem_dimension = problem.dimension
  undireted_graph = nx.Graph(problem.get_graph())
  undireted_df = nx.to_pandas_edgelist(undireted_graph)
  undireted_df_inverted = undireted_df.copy()
  column_names = undireted_df_inverted.columns.tolist()
  column_names[0], column_names[1] = column_names[1], column_names[0]
  undireted_df_inverted.columns = column_names
  undireted_df_inverted = undireted_df_inverted[column_names]
  concated_df = pd.concat([undireted_df, undireted_df_inverted]).drop_duplicates()
  filtered_df = concated_df[concated_df['source'] != concated_df['target']]

  return filtered_df

In [ ]:
def get_solution_size(cities_number: int, k_factor: float) -> int:
  return int(np.floor(k_factor * cities_number))

def get_first_city_of_solution(df: pd.DataFrame, index: int) -> int:
  sorted_df = df.sort_values('weight')
  first_city = sorted_df.iloc[index]['source']

  return first_city

def get_neighbors(df: pd.DataFrame, city: str, not_available_cities: set) -> pd.DataFrame:
  filtered_df = df[(df['source'] == city) & (df['target'] != city)]
  neighbors = filtered_df[~filtered_df['target'].isin(not_available_cities)]

  return neighbors

def get_nearest_city(neighbors: np.array) -> int:
  if neighbors.size == 0:
    return None

  nearest_index = neighbors['weight'].idxmin()
  nearest_city = neighbors.loc[nearest_index, 'target']

  return nearest_city

def generate_solution(df: pd.DataFrame, cities: np.array, solution_size: int, index: int) -> np.array:
  solution = np.zeros(solution_size, dtype=int)
  not_available_cities = set()

  if index >= solution_size:
    index = int(np.ceil(index / 2))

  first_city = get_first_city_of_solution(df, index)
  solution[0] = first_city
  not_available_cities.add(first_city)

  for i in range(1, solution_size):
    city = solution[i - 1]
    neighbors = get_neighbors(df, city, not_available_cities)
    nearest_city = get_nearest_city(neighbors)

    if nearest_city is None:
      break

    solution[i] = nearest_city
    not_available_cities.add(nearest_city)

  return solution

def check_if_solution_is_feasible(df: pd.DataFrame, solution: np.array, solution_size: int) -> bool:
  if solution.size != solution_size:
    return False

  edges = df.merge(
    pd.DataFrame({'source': solution[:-1], 'target': solution[1:]}),
    on=['source', 'target'],
    how='inner'
  )

  trips_number = len(edges)

  if trips_number != solution_size - 1:
    return False

  visited_cities = set(np.concatenate((solution[:-1], solution[1:])))

  return len(visited_cities) == solution.size

def generate_initial_population(df: pd.DataFrame, population_size: int, k_factor: float) -> np.array:
  cities = df['source'].unique()
  cities_number = cities.size
  solution_size = get_solution_size(cities_number, k_factor)
  population = np.zeros((population_size, solution_size), dtype=int)
  solution_index = 0

  for population_index in range(population_size):
    is_solution_feasible = False

    while not is_solution_feasible:
      solution = generate_solution(df, cities, solution_size, solution_index)
      is_solution_feasible = check_if_solution_is_feasible(df, solution, solution_size)

      if is_solution_feasible:
        population[population_index] = solution

      solution_index += 1

  return population

def generate_random_initial_population(df, population_size, k_factor):
  population = []
  cities = list(df['source'].unique())
  num_cities = k_factor(len(cities))

  for cromossome in range(population_size):
    is_solution_feasible = False
    while not is_solution_feasible:
      solution = np.random.choice(np.arange(1, len(cities) + 1), size=num_cities, replace=False).tolist()
      is_solution_feasible = check_if_solution_is_feasible(df, solution, k_factor)
      if is_solution_feasible:
        population.append(solution)

  return population

In [ ]:
def calculate_path_length(df: pd.DataFrame, solution: np.array) -> int:
  path_length = 0
  for i in range(len(solution) - 1):
    source = solution[i]
    target = solution[i + 1]
    edge = df[(df['source'] == source) & (df['target'] == target)]
    path_length += edge['weight'].values[0]

  return path_length

def calculate_fitness(df: pd.DataFrame, population: np.array) -> np.array:
  fitness_values = np.zeros(population.shape[0], dtype=int)

  for i, solution in enumerate(population):
    fitness_values[i] = calculate_path_length(df, solution)

  return fitness_values

In [ ]:
def roulette_selection(population: np.array, fitness_values: np.array, num_select: int) -> np.array:
  selected_indices = np.zeros(num_select, dtype=int)
  fitness_sum = np.sum(fitness_values)
  probabilities = (fitness_sum - fitness_values) / fitness_sum
  probabilities /= np.sum(probabilities)
  electist_selection_index = np.argmin(fitness_values)
  selected_indices[0] = electist_selection_index
  selected_indices[1:] = np.random.choice(len(population), size=num_select - 1, p=probabilities, replace=False)
  selected = population[selected_indices]
  selected_fitness_values = fitness_values[selected_indices]
  sorted_indices = np.argsort(selected_fitness_values)
  sorted_population = selected[sorted_indices]

  return sorted_population

In [ ]:
def swap_mutation(solution: np.array, mutation_rate: float) -> np.array:
  mutated_solution = solution.copy()
  for i in range(len(mutated_solution)):
    if np.random.random() < mutation_rate:
      j = np.random.randint(0, len(mutated_solution) - 1)
      mutated_solution[i], mutated_solution[j] = mutated_solution[j], mutated_solution[i]

  return mutated_solution

def reverse_swap_mutation(solution: np.array, mutation_rate: float) -> np.array:
  mutated_solution = solution.copy()
  for i in range(len(mutated_solution)):
    if np.random.random() < mutation_rate:
      j = np.random.randint(0, len(mutated_solution) - 1)
      start = min(i, j)
      end = max(i, j)
      mutated_solution[start:end+1] = np.flip(mutated_solution[start:end+1])

  return mutated_solution

def slide_mutation(solution: np.array, mutation_rate: float) -> np.array:
  mutated_solution = solution.copy()
  for i in range(len(mutated_solution)):
    if np.random.random() < mutation_rate:
      slide_index = np.random.randint(0, len(mutated_solution) - 1)
      slide_value = mutated_solution[i]
      mutated_solution = np.delete(mutated_solution, i)
      mutated_solution = np.insert(mutated_solution, slide_index, slide_value)

  return mutated_solution

def add_new_city_mutation(solution: np.array, mutation_rate: float) -> np.array:
  for i in range(len(solution)):
    if np.random.random() < mutation_rate:
      if i == 0:
        next_city = solution[i+1]
        closest_intermediate_city = df.query('target == @next_city and source not in @solution').sort_values('weight')
      if i == (len(solution) - 1):
        before_city = solution[i-1]
        closest_intermediate_city = df.query('source == @before_city and target not in @solution').sort_values('weight')
      else:
        before_city = solution[i-1]
        next_city = solution[i+1]
        closest_intermediate_city = df.query('(target == @before_city and source not in @solution) or (target == @next_city and source not in @solution)').groupby(['source']).sum().sort_values('weight')

      if closest_intermediate_city.shape[0] != 0:
        solution[i] = closest_intermediate_city.iloc[0].name

  return solution

def mutate(solution: np.array, mutation_rate: float) -> np.array:
  mutate_functions = [swap_mutation, reverse_swap_mutation, slide_mutation, add_new_city_mutation]
  selected_functions = np.random.choice(mutate_functions, p=mutate_functions_weights)
  mutated_solution = selected_functions(solution, mutation_rate)

  return mutated_solution

def mono_crossover(*args: np.array) -> np.array:
  return args[0]

def pairwise_crossover(*args: np.array) -> np.array:
  min_length = len(args[0])
  crossover_point = np.random.randint(1, min_length)
  child = np.concatenate((args[0][:crossover_point], args[1][crossover_point:]))

  return child

def ternary_crossover(*args: np.array) -> np.array:
  child = []

  for i in range(len(args[0])):
    if np.random.random() < 0.5:
      child.append(args[0][i])
    else:
      if np.random.random() < 0.5:
        child.append(args[1][i])
      else:
        child.append(args[2][i])

  return np.array(child)

In [ ]:
df = get_instance(selected_instances[0])
population = generate_initial_population(df, 100, k_factor_2)
fitness_values = calculate_fitness(df, population)
selected = roulette_selection(population, fitness_values, 20)
perform_crossover(selected)

KeyboardInterrupt: ignored

In [ ]:
def perform_crossover(
  population: np.array,
  num_offspring: int=80,
  crossover_fn=mono_crossover,
  k_factor=k_factor_1,
  mutation_rate: float=0.05
) -> np.array:
  selected_array = population.copy()
  offspring = []

  for i in range(num_offspring):
    is_solution_feasible = False
    while not is_solution_feasible:
      parent_indices = np.random.choice(len(selected_array), size=3, replace=False)
      parent1 = population[parent_indices[0]]
      parent2 = population[parent_indices[1]]
      parent3 = population[parent_indices[2]]
      child = crossover_fn(parent1, parent2, parent3)
      mutated_child = mutate(child, mutation_rate)
      is_solution_feasible = check_if_solution_is_feasible(df, mutated_child, k_factor)
    offspring[i] = mutated_child
    print(i)

  return offspring

In [ ]:
def get_best_individual(population, df):
  best_fitness = float('inf')
  best_individual = None

  for individual in population:
    fitness = calculate_path_length(df, individual)
    if fitness < best_fitness:
      best_fitness = fitness
      best_individual = individual

  return best_fitness, best_individual

In [ ]:
def get_diversity(population):
  unique_individuals = set(map(tuple, population))
  num_unique = len(unique_individuals)
  diversity = num_unique/len(population)

  return diversity

In [ ]:
def main(df, k_factor, debug=False):
  convergency_history = []
  diversity = 1
  initial_population = generate_initial_population(df, population_size, k_factor)
  population = initial_population
  best_fitness, best_individual = get_best_individual(population, df)
  diversity = get_diversity(population)

  if debug:
    print(f'Best fitness for originals: {best_fitness}')
    print(f'Best individual for originals: {best_individual}')
    print(f'Best fitness for originals: {diversity}')

  for generation in tqdm(range(num_generations)):
    if diversity < 0.25:
      mutation_rate = 0.75
    elif diversity < 0.5:
      mutation_rate = 0.5
    elif diversity < 0.75:
      mutation_rate = 0.25
    else:
      mutation_rate = initial_mutation_rate

    fitness_values = calculate_fitness(df, population)
    selected = roulette_selection(population, fitness_values, num_select)
    offspring = perform_crossover(selected, num_offspring=80, crossover_fn=pairwise_crossover, k_factor=k_factor, mutation_rate=mutation_rate)
    population = selected + offspring

    best_fitness, best_individual = get_best_individual(population, df)
    diversity = get_diversity(population)
    convergency_history.append(best_fitness)

    if debug:
      print(f'Best fitness for gen {generation}: {best_fitness}')
      print(f'Best individual for gen {generation}: {best_individual}')
      print(f'Diversity for gen {generation}: {diversity}')

  return best_fitness, best_individual, convergency_history

In [ ]:
# parameters
random_seed = 0
num_generations = 100
initial_mutation_rate = 0.05
mutate_functions_weights = [0.25, 0.25, 0.25, 0.25]
population_size = 100
num_select = 10
k_factor = k_factor_1

np.random.seed(random_seed)

In [ ]:
df = get_instance('tsp/burma14.tsp')
main(df, k_factor)